In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math

In [2]:
df=pd.read_csv(r'time-series-cuaca (1).csv', sep=',', header=0, low_memory=False, index_col=['Tanggal'])
df.head()

,Tavg,RH_avg,ff_x,ddd_x,Tavg_2,RH_avg_2,ff_x_2,ddd_x_2
Tanggal,,,,,,,,
23-05-2021,24.3,78.0,4,80,27.8,76.0,4,60
24-05-2021,24.1,81.0,3,210,27.5,80.0,4,150
25-05-2021,24.4,84.0,5,90,28.0,80.0,5,120
26-05-2021,NaN,NaN,6,60,27.8,84.0,5,150
27-05-2021,24.7,71.0,8,60,27.5,86.0,6,120


In [3]:
for column in df.columns:
    print(column)
    mean=df[column].mean()
    print(mean)

Tavg
24.058847736625513
RH_avg
80.01373626373626
ff_x
4.5917808219178085
ddd_x
134.1095890410959
Tavg_2
27.207810320781032
RH_avg_2
79.58659217877096
ff_x_2
5.098630136986301
ddd_x_2
138.31506849315068


In [4]:
df = df.fillna(df.mean())
df.head(5)

,Tavg,RH_avg,ff_x,ddd_x,Tavg_2,RH_avg_2,ff_x_2,ddd_x_2
Tanggal,,,,,,,,
23-05-2021,24.300000,78.000000,4,80,27.8,76.0,4,60
24-05-2021,24.100000,81.000000,3,210,27.5,80.0,4,150
25-05-2021,24.400000,84.000000,5,90,28.0,80.0,5,120
26-05-2021,24.058848,80.013736,6,60,27.8,84.0,5,150
27-05-2021,24.700000,71.000000,8,60,27.5,86.0,6,120


In [5]:
df1=df
scalers={}
for i in df.columns:
    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(df1[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    df1[i]=s_s

In [6]:
train,test = df1[:math.floor(0.75*730)], df1[math.floor(0.75*730):]

In [7]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [8]:
n_past = 90
n_future =  7
n_features = 8
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
print(X_test.shape[0],X_test.shape[1],X_test.shape[2])
print(y_test.shape[0],y_test.shape[1],y_test.shape[2])

87 90 8
87 7 8


In [9]:
# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))

# Add Conv1D layer
encoder_cnn = tf.keras.layers.Conv1D(16, kernel_size=4, activation='relu', padding='causal')(encoder_inputs)

encoder_l1 = tf.keras.layers.LSTM(16, return_sequences=True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_cnn)
encoder_states1 = encoder_outputs1[1:]

encoder_l2 = tf.keras.layers.LSTM(8, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]

dropout_rate = 0.6
dropout_layer = tf.keras.layers.Dropout(dropout_rate)
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(dropout_layer(encoder_outputs2[0]))

decoder_l1 = tf.keras.layers.LSTM(16, return_sequences=True)(decoder_inputs, initial_state=encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(8, return_sequences=True)(decoder_l1, initial_state=encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)

model_e2d2 = tf.keras.models.Model(encoder_inputs, decoder_outputs2)

model_e2d2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 90, 8)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 90, 16)       528         ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 90, 16),     2112        ['conv1d[0][0]']                 
                                 (None, 16),                                                      
                                 (None, 16)]                                                      
                                                                                              

In [10]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
# model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
# history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0447), loss=tf.keras.losses.Huber(),metrics=["mae"])
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),batch_size=32,verbose=1)#,callbacks=[reduce_lr])

Epoch 1/50
15/15 [==============================] - 13s 240ms/step - loss: 0.0349 - mae: 0.1981 - val_loss: 0.0148 - val_mae: 0.1350
Epoch 2/50
15/15 [==============================] - 2s 116ms/step - loss: 0.0116 - mae: 0.1199 - val_loss: 0.0115 - val_mae: 0.1193
Epoch 3/50
15/15 [==============================] - 2s 117ms/step - loss: 0.0108 - mae: 0.1133 - val_loss: 0.0117 - val_mae: 0.1180
Epoch 4/50
15/15 [==============================] - 2s 137ms/step - loss: 0.0106 - mae: 0.1128 - val_loss: 0.0115 - val_mae: 0.1180
Epoch 5/50
15/15 [==============================] - 2s 122ms/step - loss: 0.0106 - mae: 0.1122 - val_loss: 0.0115 - val_mae: 0.1179
Epoch 6/50
15/15 [==============================] - 2s 127ms/step - loss: 0.0106 - mae: 0.1125 - val_loss: 0.0115 - val_mae: 0.1174
Epoch 7/50
15/15 [==============================] - 2s 124ms/step - loss: 0.0106 - mae: 0.1122 - val_loss: 0.0114 - val_mae: 0.1175
Epoch 8/50
15/15 [==============================] - 2s 147ms/step - loss: 0

In [11]:
pred_e2d2=model_e2d2.predict(X_test)

3/3 [==============================] - 3s 43ms/step


In [12]:
for index,i in enumerate(train.columns):
    scaler = scalers['scaler_'+i]
    pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [13]:
from sklearn.metrics import mean_absolute_error
for index,i in enumerate(train.columns):
  print(i)
  for j in range(1,8):
    print("Day ",j,":")
    print("MAE-E2D2 : ",mean_absolute_error(y_test[:,j-1,index],pred_e2d2[:,j-1,index]))
  print()

Tavg
Day  1 :
MAE-E2D2 :  0.637811130216752
Day  2 :
MAE-E2D2 :  0.6593541726298715
Day  3 :
MAE-E2D2 :  0.6842024660658561
Day  4 :
MAE-E2D2 :  0.6953946475324959
Day  5 :
MAE-E2D2 :  0.704669715618265
Day  6 :
MAE-E2D2 :  0.7065377421762752
Day  7 :
MAE-E2D2 :  0.7051371694981367

RH_avg
Day  1 :
MAE-E2D2 :  4.53015084102236
Day  2 :
MAE-E2D2 :  4.590865124231097
Day  3 :
MAE-E2D2 :  4.645842716611665
Day  4 :
MAE-E2D2 :  4.668306986490886
Day  5 :
MAE-E2D2 :  4.682053752329158
Day  6 :
MAE-E2D2 :  4.795735501694954
Day  7 :
MAE-E2D2 :  4.903735106018768

ff_x
Day  1 :
MAE-E2D2 :  1.0736358467189746
Day  2 :
MAE-E2D2 :  1.0524714239712418
Day  3 :
MAE-E2D2 :  1.067277354755621
Day  4 :
MAE-E2D2 :  1.0567679569639008
Day  5 :
MAE-E2D2 :  1.0481925229916627
Day  6 :
MAE-E2D2 :  1.035990780797498
Day  7 :
MAE-E2D2 :  1.0539227573350929

ddd_x
Day  1 :
MAE-E2D2 :  53.8424002110273
Day  2 :
MAE-E2D2 :  56.00893454716123
Day  3 :
MAE-E2D2 :  57.85039143178655
Day  4 :
MAE-E2D2 :  58.975776

In [14]:
saved_model_path = "model/modelY.h5"

model_e2d2.save(saved_model_path)